Dans ce script, nous allons tester différents modèles de type convolution Nelle pour examiner leur performance. Nous allons ensuite tester s'ils sont robustes lorsqu'on applique une rotation à l'image d'entrée.

In [1]:
import retinoto_py as fovea
args = fovea.Params()
args

Running on MPS device (Apple Silicon/MacOS) - macos_version = 26.1
Random seed 1998 has been set.
Welcome on macOS-26.1-arm64-arm-64bit-Mach-O	User laurentperrinet Working on host obiwan.local with device mps, pytorch==2.9.1


Params(image_size=224, num_epochs=5, n_train_stop=0, seed=1998, batch_size=250, model_name='resnet50', do_scratch=False, verbose=True)

# testing a ResNEt50 model on the validation dataset

In [2]:
VAL_DATA_DIR = args.DATAROOT / 'Imagenet_full' / 'val'

In [3]:
from retinoto_py import get_loader
val_loader, class_to_idx, idx_to_class = get_loader(args, VAL_DATA_DIR)

In [4]:
from retinoto_py import load_model
model = load_model(args)
model.eval()


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
import torch
from tqdm.auto import tqdm

# --- 4. Evaluation Loop ---
correct_predictions = 0
total_predictions = 0

for images, true_labels in tqdm(val_loader, desc="Evaluating"):
    images = images.to(args.device)
    true_labels = true_labels.to(args.device)

    # Get predictions (no need for gradients)
    with torch.no_grad():
        outputs = model(images)
        _, predicted_labels = torch.max(outputs, dim=1)

    # --- Check for Correctness ---
    # The batch size is 1, so we get the first element of the tensors
    predicted_label_idx = predicted_labels[0].item()
    true_label_idx = true_labels[0].item()

    # Check if the prediction was correct for the entire batch
    # The comparison produces a tensor of booleans (True/False)
    correct_predictions_in_batch = (predicted_labels == true_labels)

    # Sum the boolean tensor to get the number of correct predictions in the batch
    # .item() extracts the number from the tensor
    correct_predictions += correct_predictions_in_batch.sum().item()

    # The total number of predictions is the batch size
    total_predictions += true_labels.size(0)


print("-" * 50)
accuracy = 100 * correct_predictions / total_predictions
print(f"Evaluation complete.")
print(f"Accuracy: {correct_predictions}/{total_predictions} ({accuracy:.2f}%)")

Evaluating:   0%|          | 0/200 [00:00<?, ?it/s]